# PROGRAMA DE CIENCIAS DE LOS DATOS 
# **Curso: Big Data**
## **PROYECTO FINAL:**

## <font color='red'>Machine Learning con datos en PostgreSQL</font>. 


#### **Profesor: MSc. Felipe Meza Obando**


#### Alumnos: 
  
####  **Lester Salazar Viales.**
####  **Randal Salazar Viales.**


### Objetivo del Proyecto

Se efectuará un Análisis de datos de Machine Learning, a una base de datos que se encuentra ubicada en una **BD PostgreSQL**.

Para la BD a emplear, se efectuará un **análisis predictorio**, para tratar de **determinar la cantidad de días de vacaciones que  posee cada empleado**, de acuerdo a las features existentes en la BD.

### Creación de SparkSession

In [1]:
import findspark
findspark.init('C:\spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "C:\Spark\jdbcdriver\postgresql-42.2.9.jar") \
    .config("spark.executor.extraClassPath", "C:\Spark\jdbcdriver\postgresql-42.2.9.jar") \
    .getOrCreate()

### Conexión a BD PostgreSQL mediante Spark

### Creación del DataFrame de datos de BD empleada

- **Dataframe de Tabla Empleados**

In [2]:
# Reading single DataFrame in Spark by retrieving all rows from a DB table.
empleados_df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/rhdb") \
    .option("user", "postgres") \
    .option("password", "bd2019%") \
    .option("dbtable", "empleados") \
    .load()

empleados_df.show(5)

+----------+---------+---------+--------------------+------------+---------+----------+------------+--------------------+-------------+----------------+------------+---------------+----------+----+
|    Cedula|Apellido1|Apellido2|              Nombre|Cod_Planilla|Provincia|Cod_Canton|Cod_Distrito|           Direccion|Fecha_Ingreso|Fecha_Nacimiento|Fecha_Salida|Jornada_Trabajo|Cod_Puesto|Sexo|
+----------+---------+---------+--------------------+------------+---------+----------+------------+--------------------+-------------+----------------+------------+---------------+----------+----+
|0304810504|   ABARCA|  MIRANDA|       SUSANA ARIELA|           Q|        3|        02|         000|CARTAGO, PARAISO,...|   2017-05-03|      1994-09-14|        null|              A|   ASB2.33|   F|
|0304340108|   ABARCA|    ARAYA|YENIFFER DE LOS A...|           Q|        3|        08|         010|RESIDENCIAL LOS Z...|   2007-12-03|      1989-01-25|        null|              A|   ASB2.33|   F|
|011297084

Visualización del esquema del DataFrame:

In [3]:
empleados_df.printSchema()

root
 |-- Cedula: string (nullable = true)
 |-- Apellido1: string (nullable = true)
 |-- Apellido2: string (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Cod_Planilla: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Cod_Canton: string (nullable = true)
 |-- Cod_Distrito: string (nullable = true)
 |-- Direccion: string (nullable = true)
 |-- Fecha_Ingreso: date (nullable = true)
 |-- Fecha_Nacimiento: date (nullable = true)
 |-- Fecha_Salida: date (nullable = true)
 |-- Jornada_Trabajo: string (nullable = true)
 |-- Cod_Puesto: string (nullable = true)
 |-- Sexo: string (nullable = true)



- **Dataframe de Tabla Vacaciones**

In [4]:
# Reading single DataFrame in Spark by retrieving all rows from a DB table.
emp_vacac_df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/rhdb") \
    .option("user", "postgres") \
    .option("password", "bd2019%") \
    .option("dbtable", "empleado_vacaciones") \
    .load()

emp_vacac_df.show(10)

+----------+------------------+-------------+--------------------+---------------+---------------+
|    Cedula|Periodo_vacaciones|Fecha_Derecho|        Dias_Derecho|Dias_Disfrutado|Dias_Adelantado|
+----------+------------------+-------------+--------------------+---------------+---------------+
|0118020085|              2020|   2019-05-22|15.00000000000000...|          0E-18|          0E-18|
|0110160466|              2020|   2001-07-20|30.00000000000000...|          0E-18|          0E-18|
|0116260447|              2020|   2018-11-23|15.00000000000000...|          0E-18|          0E-18|
|0115470697|              2020|   2017-12-01|15.00000000000000...|          0E-18|          0E-18|
|0304350400|              2020|   2016-05-02|15.00000000000000...|          0E-18|          0E-18|
|0304810504|              2020|   2017-05-03|15.00000000000000...|          0E-18|          0E-18|
|0117110780|              2020|   2016-12-26|15.00000000000000...|          0E-18|          0E-18|
|030515010

Visualización del esquema del DataFrame:

In [5]:
emp_vacac_df.printSchema()

root
 |-- Cedula: string (nullable = true)
 |-- Periodo_vacaciones: string (nullable = true)
 |-- Fecha_Derecho: date (nullable = true)
 |-- Dias_Derecho: decimal(38,18) (nullable = true)
 |-- Dias_Disfrutado: decimal(38,18) (nullable = true)
 |-- Dias_Adelantado: decimal(38,18) (nullable = true)



- **Dataframe de Tabla Empleado-Ingreso-Egreso**

In [6]:
# Reading single DataFrame in Spark by retrieving all rows from a DB table.
emp_ingr_egre_df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/rhdb") \
    .option("user", "postgres") \
    .option("password", "bd2019%") \
    .option("dbtable", "empleado_ingreso_egreso") \
    .load()

emp_ingr_egre_df.show(5)

+----------+---------------------+----------------+---------------------+-------------------+--------------------+------------------------+----------+----------+---------+--------+
|    Cedula|Cod_Concepto_Salarial|Fecha_Rige_Desde|Estado_Ingreso_Egreso|Tipo_Ingreso_Egreso|       Monto_Aplicar|Categoria_Ingreso_Egreso|Forma_Pago|Cod_Puesto|Cod_Plaza|Cod_Area|
+----------+---------------------+----------------+---------------------+-------------------+--------------------+------------------------+----------+----------+---------+--------+
|0118020085|                  041|      2018-01-01|                    A|                  I|257627.2000000000...|                     SAL|         C|   ASB2.33|     3227|     051|
|0110160466|                  001|      2018-01-01|                    A|                  I|441407.0400000000...|                     SAL|         C|   PRC8.03|     2847|     051|
|0116260447|                  041|      2018-01-01|                    A|                  I|30

Visualización del esquema del DataFrame:

In [7]:
emp_ingr_egre_df.printSchema()

root
 |-- Cedula: string (nullable = true)
 |-- Cod_Concepto_Salarial: string (nullable = true)
 |-- Fecha_Rige_Desde: date (nullable = true)
 |-- Estado_Ingreso_Egreso: string (nullable = true)
 |-- Tipo_Ingreso_Egreso: string (nullable = true)
 |-- Monto_Aplicar: decimal(38,18) (nullable = true)
 |-- Categoria_Ingreso_Egreso: string (nullable = true)
 |-- Forma_Pago: string (nullable = true)
 |-- Cod_Puesto: string (nullable = true)
 |-- Cod_Plaza: string (nullable = true)
 |-- Cod_Area: string (nullable = true)



- **Unión de los 3 dataframes para tener todas las columnas necesarias pra efectuar el Análisis en un solo dataframe**

- Unión de dataframes: **empleados_df + emp_vacac_df**

In [8]:
join12 = empleados_df.join(emp_vacac_df, empleados_df.Cedula == emp_vacac_df.Cedula)

In [9]:
join12.printSchema()

root
 |-- Cedula: string (nullable = true)
 |-- Apellido1: string (nullable = true)
 |-- Apellido2: string (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Cod_Planilla: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Cod_Canton: string (nullable = true)
 |-- Cod_Distrito: string (nullable = true)
 |-- Direccion: string (nullable = true)
 |-- Fecha_Ingreso: date (nullable = true)
 |-- Fecha_Nacimiento: date (nullable = true)
 |-- Fecha_Salida: date (nullable = true)
 |-- Jornada_Trabajo: string (nullable = true)
 |-- Cod_Puesto: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Cedula: string (nullable = true)
 |-- Periodo_vacaciones: string (nullable = true)
 |-- Fecha_Derecho: date (nullable = true)
 |-- Dias_Derecho: decimal(38,18) (nullable = true)
 |-- Dias_Disfrutado: decimal(38,18) (nullable = true)
 |-- Dias_Adelantado: decimal(38,18) (nullable = true)



- Comparación de valores de campo Cédula de ambos dataframes anteriores en el nuevo dataframe

In [10]:
join12.select(empleados_df['Cedula'], emp_vacac_df['Cedula']).show(2)

+----------+----------+
|    Cedula|    Cedula|
+----------+----------+
|0110530432|0110530432|
|0111710991|0111710991|
+----------+----------+
only showing top 2 rows



- Eliminación de columnas demás del dataframe nuevo

In [11]:
data12 = join12.select([empleados_df['Cedula'], 'Cod_Planilla', 'Provincia','Fecha_Ingreso', 'Fecha_Nacimiento', 
                        'Jornada_Trabajo', 'Cod_Puesto', 'Sexo', 'Dias_Derecho', 'Dias_Disfrutado'])

In [12]:
data12.show(2)

+----------+------------+---------+-------------+----------------+---------------+----------+----+--------------------+---------------+
|    Cedula|Cod_Planilla|Provincia|Fecha_Ingreso|Fecha_Nacimiento|Jornada_Trabajo|Cod_Puesto|Sexo|        Dias_Derecho|Dias_Disfrutado|
+----------+------------+---------+-------------+----------------+---------------+----------+----+--------------------+---------------+
|0110530432|           Q|        3|   2005-11-21|      1979-01-05|              A|  HPRC8.10|   F|20.00000000000000...|          0E-18|
|0111710991|           Q|        6|   2006-10-02|      1983-05-14|              A|   PRC4.11|   M|20.00000000000000...|          0E-18|
+----------+------------+---------+-------------+----------------+---------------+----------+----+--------------------+---------------+
only showing top 2 rows



In [13]:
data12.printSchema()

root
 |-- Cedula: string (nullable = true)
 |-- Cod_Planilla: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Fecha_Ingreso: date (nullable = true)
 |-- Fecha_Nacimiento: date (nullable = true)
 |-- Jornada_Trabajo: string (nullable = true)
 |-- Cod_Puesto: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Dias_Derecho: decimal(38,18) (nullable = true)
 |-- Dias_Disfrutado: decimal(38,18) (nullable = true)



- Unión de dataframes: **join12_df + emp_ingr_egre_df**

In [14]:
join23 = data12.join(emp_ingr_egre_df, data12.Cedula == emp_ingr_egre_df.Cedula)

In [15]:
join23.printSchema()

root
 |-- Cedula: string (nullable = true)
 |-- Cod_Planilla: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Fecha_Ingreso: date (nullable = true)
 |-- Fecha_Nacimiento: date (nullable = true)
 |-- Jornada_Trabajo: string (nullable = true)
 |-- Cod_Puesto: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Dias_Derecho: decimal(38,18) (nullable = true)
 |-- Dias_Disfrutado: decimal(38,18) (nullable = true)
 |-- Cedula: string (nullable = true)
 |-- Cod_Concepto_Salarial: string (nullable = true)
 |-- Fecha_Rige_Desde: date (nullable = true)
 |-- Estado_Ingreso_Egreso: string (nullable = true)
 |-- Tipo_Ingreso_Egreso: string (nullable = true)
 |-- Monto_Aplicar: decimal(38,18) (nullable = true)
 |-- Categoria_Ingreso_Egreso: string (nullable = true)
 |-- Forma_Pago: string (nullable = true)
 |-- Cod_Puesto: string (nullable = true)
 |-- Cod_Plaza: string (nullable = true)
 |-- Cod_Area: string (nullable = true)



- Comparación de valores de campo Cédula de ambos dataframes anteriores en el nuevo dataframe

In [16]:
join23.select(data12['Cedula'], emp_ingr_egre_df['Cedula']).show(2)

+----------+----------+
|    Cedula|    Cedula|
+----------+----------+
|0110530432|0110530432|
|0111710991|0111710991|
+----------+----------+
only showing top 2 rows



- Eliminación de columnas demás del dataframe nuevo

In [17]:
data_df = join23.select([data12['Cedula'], 'Cod_Planilla', 'Provincia','Fecha_Ingreso', 'Fecha_Nacimiento', 'Jornada_Trabajo', 
                         data12['Cod_Puesto'], 'Sexo', 'Dias_Derecho', 'Dias_Disfrutado', 'Cod_Concepto_Salarial', 
                         'Monto_Aplicar', 'Cod_Area'])

- Dataframe con columnas a utilizar:

In [18]:
data_df.printSchema()

root
 |-- Cedula: string (nullable = true)
 |-- Cod_Planilla: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Fecha_Ingreso: date (nullable = true)
 |-- Fecha_Nacimiento: date (nullable = true)
 |-- Jornada_Trabajo: string (nullable = true)
 |-- Cod_Puesto: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Dias_Derecho: decimal(38,18) (nullable = true)
 |-- Dias_Disfrutado: decimal(38,18) (nullable = true)
 |-- Cod_Concepto_Salarial: string (nullable = true)
 |-- Monto_Aplicar: decimal(38,18) (nullable = true)
 |-- Cod_Area: string (nullable = true)



In [19]:
data_df.show(2)

+----------+------------+---------+-------------+----------------+---------------+----------+----+--------------------+---------------+---------------------+--------------------+--------+
|    Cedula|Cod_Planilla|Provincia|Fecha_Ingreso|Fecha_Nacimiento|Jornada_Trabajo|Cod_Puesto|Sexo|        Dias_Derecho|Dias_Disfrutado|Cod_Concepto_Salarial|       Monto_Aplicar|Cod_Area|
+----------+------------+---------+-------------+----------------+---------------+----------+----+--------------------+---------------+---------------------+--------------------+--------+
|0110530432|           Q|        3|   2005-11-21|      1979-01-05|              A|  HPRC8.10|   F|20.00000000000000...|          0E-18|                  041|855421.4700000000...|     001|
|0111710991|           Q|        6|   2006-10-02|      1983-05-14|              A|   PRC4.11|   M|20.00000000000000...|          0E-18|                  041|464802.1700000000...|     051|
+----------+------------+---------+-------------+-----------

- **Cálculo de cantidad de días de vacaciones disponibles de cada empleado**

El cálculo de días de vacaciones viene en función de las columnas **Dias_Derecho** y **Dias_Disfrutado**.

Se calcula de la siguiente forma:

    Si Dias_Disfrutado < 0, Vacaciones = Dias_Disfutado, de lo contrario
                            Vacaciones = (Dias_Derecho - Dias_Disfrutado

Por tanto, si Vacaciones < 0, la persona a realizado más días de vacaciones de las que tiene disponible.